In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import networkx as nx

from efb import SimpleEFB

In [2]:
data = pd.read_csv("./data/boston.csv")
X = data.drop("target", axis=1)
y = data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_onehot = pd.concat([X, pd.get_dummies(data["CHAS"], prefix="CHAS", dtype=int)], axis=1)
# X_onehot.drop("CHAS", axis=1, inplace=True)

# X_onehot = pd.concat([X, pd.get_dummies(data["RAD"], prefix="RAD", dtype=int)], axis=1)
# X_onehot.drop("RAD", axis=1, inplace=True)
# X_onehot['ADD'] = 1

In [ ]:
def _create_weighted_feature_graph(X: pd.DataFrame) -> nx.Graph:
    G = nx.Graph()
    feats = list(X.columns)
    for feat in feats:
        G.add_node(feat)

    for i, feature_i in enumerate(feats):
        for j, feature_j in enumerate(feats):
            # 重複して評価しないため
            if i < j:
                # nonzeroかつ同じ値の数をカウント（衝突数）
                # TODO: numerical featureの場合はbinningしてからカウントする
                non_zero_mask = (X[feature_i] != 0) & (X[feature_j] != 0)
                conflicts = (X[feature_i][non_zero_mask] == X[feature_j][non_zero_mask]).sum()

                # 衝突がある場合のみweightを衝突数とするedgeを追加
                if conflicts > 0:
                    G.add_edge(feature_i, feature_j, weight=conflicts)
    return G

def _greedy_bundling(G: nx.Graph, total_sample_cnt, threshold=None) -> dict:
    # https://github.com/microsoft/LightGBM/issues/4114#issuecomment-813201652
    if threshold is None:
        threshold = int(total_sample_cnt / 10000)

    bundles = {}
    # (edge数 * weight) でソート
    sortedNodes = sorted(G.degree(weight='weight'), key=lambda x: x[1], reverse=True)

    # より多くの衝突がある特徴量から処理を行う
    for feat, degree in sortedNodes:
        needNew = True
        for i in range(len(bundles)):
            conflicts = 0
            # 既存のバンドル（bundles[i]）に含まれる全ての特徴量との衝突数を合計する
            for f in bundles[i]:
                if f in G[feat]:
                    conflicts += G[feat][f]['weight']

            # 値の衝突が閾値以下であれば、マージする
            # 閾値以上であれば、新しいバンドルを追加する
            if conflicts <= threshold:
                bundles[i].append(feat)
                # bundle内で最も衝突が小さいものにマージするのではなく、早い者勝ちでマージする
                needNew = False
                break
        if needNew:
            idx = len(bundles)
            bundles[idx] = [feat]
    
    return bundles
    
def _get_bins(X: pd.Series, max_bin) -> int:
    unique_cnt = X.nunique()
    return min(unique_cnt, max_bin)

def _merge_exclusive_feature(X: pd.DataFrame, bundles: dict, max_bin=255) -> pd.DataFrame:
    df = pd.DataFrame()

    X = X.copy()

    X = X.reset_index(drop=True)

    for i, feats in enumerate(bundles.values()):
        bin_ranges = {}
        total_bin = 0

        # offsetを計算
        for feat in feats:
            # 各特徴量のbin数を加算していくことで、offsetを計算する
            total_bin += _get_bins(X[feat], max_bin)
            bin_ranges[feat] = total_bin

        bin = pd.Series(np.zeros(len(X), dtype=int))
        for feat in feats:
            bin_df = pd.cut(X[feat], bins=_get_bins(X[feat], max_bin), labels=False)
            zero_mask = bin_df == 0

            # bin値が0の場合は、offsetを加算しない
            # そもそもここで加算している特徴量同士は、同時にnonzero値を取らないことが前提であるため、
            # offsetの加算により、どの特徴量のbin値かが一意に特定できる。0にoffsetを加算すると、この一意性が失われる。
            bin_df += bin_ranges[feat]
            bin_df[zero_mask] = 0

            bin += bin_df

        df[i] = bin

    return df


In [ ]:
G = _create_weighted_feature_graph(X_train)
bundles = _greedy_bundling(G, X_train.shape[0])
X = _merge_exclusive_feature(X_train, bundles)

In [10]:
from sklearn.model_selection import ParameterGrid

def simple_hyperparameter_search(X_train, y_train, X_test, y_test):
    param = {
        "n_trees": [10, 100, 1000],
        "learning_rate": [0.1, 0.01, 0.001],
        "max_depth": range(3, 100, 5),
    }
    param_grid = ParameterGrid(param)

    best_score = 1000
    best_param = None
    total = len(param_grid)
    for i, p in enumerate(param_grid):
        if i % 30 == 0:
            print(f"{i}/{total}")
        efb = SimpleEFB(**p)
        efb.fit(X_train, y_train)
        y_pred = efb.predict(X_test)
        score = mean_squared_error(y_test, y_pred)

        if score < best_score:
            best_score = score
            best_param = p
    
    return best_param, best_score

In [11]:
simple_hyperparameter_search(X_train, y_train, X_test, y_test)

0/180
30/180
60/180
90/180
120/180
150/180


({'learning_rate': 0.001, 'max_depth': 8, 'n_trees': 10}, 75.08955622266137)

In [62]:
efb = SimpleEFB(n_trees=700, learning_rate=0.002, max_depth=9, random_state=42, max_bin=4)
efb.fit(X_train, y_train)
y_pred_train = efb.predict(X_train)
y_pred_test = efb.predict(X_test)

print(f"Train MSE: {mean_squared_error(y_train, y_pred_train):.4f}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred_test):.4f}")


Train MSE: 10.4592
Test MSE: 22.6830
